# Application: Track Vehicle Trajectory

#### References

1. **Extended Kalman filter:** https://en.wikipedia.org/wiki/Extended_Kalman_filter
2. **Extended Kalman Filter Lecture Notes:** https://www.cs.cmu.edu/~motionplanning/papers/sbp_papers/kalman/ekf_lecture_notes.pdf

## Track Vehicle Trajectory

In this application we will be recursively estimating a vehicle trajectory using available measurements and motion model. 
The vehicle is equipped with a LIDAR sensor, which returns range and bearing measurements corresponding to individual landmarks in the environment. The correspondences between landmarks and their global positions are assumed to be known beforehand. We also assume knowledge about which measurement belongs to which landmark.

### Motion and measurement models

#### Motion model

The motion model recieves the linear and angular velocity odometry readings as inputs, and outputs the state (2D pose) of the vehicle:


\begin{align}
\mathbf{x}_{k} &= \mathbf{x}_{k-1} + T
\begin{bmatrix}
\cos\theta_{k-1} &0 \\
\sin\theta_{k-1} &0 \\
0 &1
\end{bmatrix}
\left(
\begin{bmatrix}
v_k \\
\omega_k
\end{bmatrix}
+ \mathbf{w}_k
\right)
\, , \, \, \, \, \, \mathbf{w}_k = \mathcal{N}\left(\mathbf{0}, \mathbf{Q}\right)
\end{align}

- $\mathbf{x}_k = \left[ x \, y \, \theta \right]^T$ is the current 2D pose of the vehicle
- $v_k$ and $\omega_k$ are the linear and angular velocity odometry readings, which we are using as inputs

The process noise $\mathbf{w}_k$ has a normal distribution with a constant covariance $\mathbf{Q}$.

#### Measurement model

The measurement model relates the current pose of the vehicle to the LIDAR range and bearing measurements $\mathbf{y}^l_k = \left[r \, \phi \right]^T$.


\begin{align}
\mathbf{y}^l_k =
\begin{bmatrix}
\sqrt{(x_l - x_k - d\cos\theta_{k})^2 + (y_l - y_k - d\sin\theta_{k})^2} \\
atan2\left(y_l - y_k - d\sin\theta_{k},x_l - x_k - d\cos\theta_{k}\right) - \theta_k
\end{bmatrix}
+
\mathbf{n}^l_k
\, , \, \, \, \, \, \mathbf{n}^l_k = \mathcal{N}\left(\mathbf{0}, \mathbf{R}\right)
\end{align}



- $x_l$ and $y_l$ are the ground truth coordinates of the landmark $l$
- $x_k$ and $y_k$ and $\theta_{k}$ represent the current pose of the vehicle
- $d$ is the known distance between robot center and laser rangefinder (LIDAR)

The landmark measurement noise $\mathbf{n}^l_k$ has a normal distribution with a constant covariance $\mathbf{R}$.

### Getting started

Since the above models are nonlinear, we recommend using the Extended Kalman Filter (EKF) as the state estimator.
Specifically, you will need to provide code implementing the following steps:

- the prediction step, which uses odometry measurements and above motion model to provide a state and covariance estimate at a given timestep
- the correction step, which uses the range and bearing measurements provided by the LIDAR to correct the estimates


### Unpack data
-----

First, let's unpack the available data:

In [1]:
# Need these so that Jupyter can find the simulator types
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

In [33]:
import pickle
import numpy as np
from numpy.linalg import inv
import matplotlib.pyplot as plt

with open('data/vehicle_trajectory/data.pickle', 'rb') as f:
    data = pickle.load(f)

t = data['t']  # timestamps [s]

# input signal
v = data['v']  # translational velocity input [m/s]
om = data['om']  # rotational velocity input [rad/s]

# bearing and range measurements, LIDAR constants
b = data['b']  # bearing to each landmarks center in the frame attached to the laser [rad]
r = data['r']  # range measurements [m]
l = data['l']  # x,y positions of landmarks [m]
d = data['d']  # distance between robot center and laser rangefinder [m]

---

**Remark**

Note that distance from the LIDAR frame to the robot center is provided and loaded as an array into the `d` variable.

---

### Ground truth
-----

If available, it is useful to plot the ground truth position and orientation out before starting the assignment.






<table><tr>
<td> <img src="img/gtruth.png" alt="Ground Truth" width="350"/> </td>
<td> <img src="img/gtruth2.png" alt="Ground Truth" width="350"/> </td>
</tr></table>



Notice that the orientation values are wrapped to the $\left[-\pi,\pi\right]$ range in radians.

### Initializing parameters
-----

Now that our data is loaded, we can start getting things ready for our solver. One of the
most important aspects of a filter is setting the input and measurement noise covariance matrices, as well as the initial state and covariance values. We set the values here:

In [3]:
v_var = 0.01  # translation velocity variance  
om_var = 0.01  # rotational velocity variance 
r_var = 0.1  # range measurements variance
b_var = 0.1  # bearing measurement variance

Q_km = np.diag([v_var, om_var]) # input noise covariance 
cov_y = np.diag([r_var, b_var])  # measurement noise covariance 

x_est = np.zeros([len(v), 3])  # estimated states, x, y, and theta
P_est = np.zeros([len(v), 3, 3])  # state covariance matrices

x_est[0] = np.array([0.0, 0.0, 0.0]) # initial state
P_est[0] = np.diag([1, 1, 0.1]) # initial state covariance

---

**Remark**

Remember that it is neccessary to tune the measurement noise variances `r_var`, `b_var` in order for the filter to perform well.

---

In order for the orientation estimates to coincide with the bearing measurements, it is neccessary to wrap all estimated $\theta$ values to the $(-\pi , \pi)$ range.

In [4]:
# Wraps angle to [-pi,pi] range
def wraptopi(x):
    if x > np.pi:
        x = x - (np.floor(x / (2 * np.pi)) + 1) * 2 * np.pi
    elif x < -np.pi:
        x = x + (np.floor(x / (-2 * np.pi)) + 1) * 2 * np.pi
    return x


### Correction step
-----

First, let's implement the measurement update function, which takes an available landmark measurement $l$ and updates the current state estimate $\mathbf{\check{x}}_k$. For each landmark measurement in a given timestep $k$ you should implement the following steps:

- Compute the measurement model Jacobians at $\mathbf{\check{x}}_{k}$


\begin{align}
\mathbf{y}^l_k = &\mathbf{h}(\mathbf{x}_{k}, \mathbf{n}^l_k) \\\\
\mathbf{H}_{k} = \frac{\partial \mathbf{h}}{\partial \mathbf{x}_{k}}\bigg|_{\mathbf{\check{x}}_{k},0}& \, , \, \, \, \,
\mathbf{M}_{k} = \frac{\partial \mathbf{h}}{\partial \mathbf{n}_{k}}\bigg|_{\mathbf{\check{x}}_{k},0} \, .
\end{align}

- Compute the Kalman Gain

\begin{align}
\mathbf{K}_k &= \mathbf{\check{P}}_k \mathbf{H}_k^T \left(\mathbf{H}_k \mathbf{\check{P}}_k \mathbf{H}_k^T + \mathbf{M}_k \mathbf{R}_k \mathbf{M}_k^T \right)^{-1} 
\end{align}


- Correct the predicted state


\begin{align}
\mathbf{\check{y}}^l_k &= \mathbf{h}\left(\mathbf{\check{x}}_k, \mathbf{0}\right) \\
\mathbf{\hat{x}}_k &= \mathbf{\check{x}}_k + \mathbf{K}_k \left(\mathbf{y}^l_k - \mathbf{\check{y}}^l_k\right)
\end{align}


- Correct the covariance


\begin{align}
\mathbf{\hat{P}}_k &= \left(\mathbf{I} - \mathbf{K}_k \mathbf{H}_k \right)\mathbf{\check{P}}_k
\end{align}

In [29]:
def compute_meas_jacobian(lk, d_k, x_check):
    
    x_k = x_check[0]
    x_l = lk[0]
    y_k = x_check[1]
    y_l = lk[1] 
    theta_k = x_check[2]
    d = d_k
    
    # diff(sqrt((x_l - x_k - d*cos(theta_k))**2 + (y_l - y_k - d*sin(theta_k))**2), x_k)
    # (d*cos(theta_k) + x_k - x_l)/sqrt((-d*sin(theta_k) - y_k + y_l)**2 + (-d*cos(theta_k) - x_k + x_l)**2)
    denom = np.sqrt((-d*np.sin(theta_k) - y_k + y_l)**2 + (-d*np.cos(theta_k) - x_k +x_l)**2)
    partial_x_k_0 = (d*np.cos(theta_k) + x_k - x_l)/denom
    
    # diff(sqrt((x_l - x_k - d*cos(theta_k))**2 + (y_l - y_k - d*sin(theta_k))**2), y_k)
    # (d*sin(theta_k) + y_k - y_l)/sqrt((-d*sin(theta_k) - y_k + y_l)**2 + (-d*cos(theta_k) - x_k + x_l)**2)
    partial_y_k_0 = (d*np.sin(theta_k) + y_k - y_l)/denom
    
    # diff(sqrt((x_l - x_k - d*cos(theta_k))**2 + (y_l - y_k - d*sin(theta_k))**2), theta_k)
    # (-d*(-d*sin(theta_k) - y_k + y_l)*cos(theta_k) + d*(-d*cos(theta_k) - x_k + x_l)*sin(theta_k))/
    # sqrt((-d*sin(theta_k) - y_k + y_l)**2 + (-d*cos(theta_k) - x_k + x_l)**2)
    
    nom = (-d*(-d*np.sin(theta_k) - y_k + y_l)*np.cos(theta_k) + d*(-d*np.cos(theta_k) - x_k + x_l)*np.sin(theta_k))
    partial_theta_k_0 = nom/denom
    
    # diff(atan2(y_l - y_k -d*sin(theta_k), x_l - x_k - d*cos(theta_k)) - theta_k, x_k)
    # -(d*sin(theta_k) + y_k - y_l)/((-d*sin(theta_k) - y_k + y_l)**2 + (-d*cos(theta_k) - x_k + x_l)**2)
    nom = -(d*np.sin(theta_k) + y_k - y_l)
    denom = ((-d*np.sin(theta_k) - y_k + y_l)**2 + (-d*np.cos(theta_k) - x_k + x_l)**2)
    partial_x_k_1 = nom /denom
    
    # diff(atan2(y_l - y_k -d*sin(theta_k), x_l - x_k - d*cos(theta_k)) - theta_k, y_k)
    # -(-d*cos(theta_k) - x_k + x_l)/((-d*sin(theta_k) - y_k + y_l)**2 + (-d*cos(theta_k) - x_k + x_l)**2)
    nom = -(-d*np.cos(theta_k) - x_k + x_l)
    denom = ((-d*np.sin(theta_k) - y_k + y_l)**2 + (-d*np.cos(theta_k) - x_k + x_l)**2)
    partial_y_k_1 = nom /denom
    
    # diff(atan2(y_l - y_k -d*sin(theta_k), x_l - x_k - d*cos(theta_k)) - theta_k, theta_k)
    # d*(d*sin(theta_k) + y_k - y_l)*sin(theta_k)/
    #((-d*sin(theta_k) - y_k + y_l)**2 + (-d*cos(theta_k) - x_k + x_l)**2) - d*(-d*cos(theta_k) - x_k + x_l)*cos(theta_k)/((-d*sin(theta_k) - y_k + y_l)**2 + (-d*cos(theta_k) - x_k + x_l)**2) - 1
    nom = d*(d*np.sin(theta_k) + y_k - y_l)*np.sin(theta_k)
    denom_1 = ((-d*np.sin(theta_k) - y_k + y_l)**2 + (-d*np.cos(theta_k) - x_k + x_l)**2) - 1
    denom = ((-d*np.sin(theta_k) - y_k + y_l)**2 + (-d*np.cos(theta_k) - x_k + x_l)**2) - d*(-d*np.cos(theta_k) - x_k + x_l)*np.cos(theta_k)/denom_1
    partial_theta_k_1 = nom / denom
    H_k = np.array([[partial_x_k_0, partial_y_k_0, partial_theta_k_0], 
                    [partial_x_k_1, partial_y_k_1, partial_theta_k_1 ]])
    
    M_k = np.array([1.0])
    return H_k, M_k
    

In [30]:
def correct_variance(K, H, P_check):
    
    K_times_H = np.dot(K, H)
    I = np.identity(K_times_H.shape[0])
    P_k = np.dot(I - K_times_H, P_check)
    return P_k
    

In [31]:
def measurement_update(lk, rk, bk,  d, P_check, x_check):
    
    # 1. Compute measurement model Jacobian
    H_k, M_k = compute_meas_jacobian(lk=lk, d_k=d, x_check=x_check)

    # 2. Compute Kalman Gain
    H_k_T = H_k.T
    H_k_P_check_H_T = np.dot(H_k, np.dot(P_check, H_k_T))
    M_k_R_k_M_T = np.dot(M_k, np.dot(rk, M_k.T))
    K_k = np.dot(np.dot(P_check, H_k.T),inv(H_k_P_check_H_T + M_k_R_k_M_T))
    
    y_check = np.ones(x_check.shape)

    # 3. Correct predicted state (remember to wrap the angles to [-pi,pi])
    x_check = x_check + K_k*(y_check)
    
    x_check[2] = wraptopi(x_check[2])

    # 4. Correct covariance
    P_check = correct_variance(K, H_k, P_check)

    return x_check, P_check


### Prediction step
-----

Now, implement the main filter loop, implementing the prediction step of the EKF using the provided motion model:




\begin{align}
\mathbf{\check{x}}_k &= \mathbf{f}\left(\mathbf{\hat{x}}_{k-1}, \mathbf{u}_{k-1}, \mathbf{0} \right) \\
\mathbf{\check{P}}_k &= \mathbf{F}_{k-1}\mathbf{\hat{P}}_{k-1}\mathbf{F}_{k-1}^T + \mathbf{L}_{k-1}\mathbf{Q}_{k-1}\mathbf{L}_{k-1}^T \, .
\end{align}



Where



\begin{align}
\mathbf{F}_{k-1} = \frac{\partial \mathbf{f}}{\partial \mathbf{x}_{k-1}}\bigg|_{\mathbf{\hat{x}}_{k-1},\mathbf{u}_{k},0}  \, , \, \, \, \,
\mathbf{L}_{k-1} = \frac{\partial \mathbf{f}}{\partial \mathbf{w}_{k}}\bigg|_{\mathbf{\hat{x}}_{k-1},\mathbf{u}_{k},0} \, .
\end{align}

In [32]:
#### 5. Main Filter Loop #######################################################################
for k in range(1, len(t)):  # start at 1 because we've set the initial prediciton

    delta_t = t[k] - t[k - 1]  # time step (difference between timestamps)

    # 1. Update state with odometry readings (remember to wrap the angles to [-pi,pi])
    x_check = np.zeros(3)

    # 2. Motion model jacobian with respect to last state
    F_km = np.zeros([3, 3])

    # 3. Motion model jacobian with respect to noise
    L_km = np.zeros([3, 2])
    
    print("shape P_est: ",P_est[0].shape )

    # 4. Propagate uncertainty
    P_check = np.dot(F_km, np.dot(P_est[0], F_km.T)) + np.dot(L_km, np.dot(Q_km, L_km.T))

    # 5. Update state estimate using available landmark measurements
    for i in range(len(r[k])):
        x_check, P_check = measurement_update(l[i], r[k, i], b[k, i], P_check=P_check, x_check=x_check, d=d)

    # Set final state predictions for timestep
    x_est[k, 0] = x_check[0]
    x_est[k, 1] = x_check[1]
    x_est[k, 2] = x_check[2]
    P_est[k, :, :] = P_check

shape P_est:  (3, 3)


NameError: name 'inv' is not defined

Let's plot the resulting state estimates:

In [ ]:
e_fig = plt.figure()
ax = e_fig.add_subplot(111)
ax.plot(x_est[:, 0], x_est[:, 1])
ax.set_xlabel('x [m]')
ax.set_ylabel('y [m]')
ax.set_title('Estimated trajectory')
plt.show()

e_fig = plt.figure()
ax = e_fig.add_subplot(111)
ax.plot(t[:], x_est[:, 2])
ax.set_xlabel('Time [s]')
ax.set_ylabel('theta [rad]')
ax.set_title('Estimated trajectory')
plt.show()

Are you satisfied wth your results? The resulting trajectory should closely resemble the ground truth, with minor "jumps" in the orientation estimate due to angle wrapping. If this is the case, run the code below to produce your solution file.

In [ ]:
with open('submission.pkl', 'wb') as f:
    pickle.dump(x_est, f, pickle.HIGHEST_PROTOCOL)